# Verificação formal de programas: a metodologia de Floyd-Hoare.

A *verificação formal de programas* tem por objectivo garantir que um programa satisfaz a sua especificação.
Uma *especificação* é um modelo dum sistema que contém uma descrição do seu comportamento desejado ("*o que*" deve ser implementado, por oposição a "*como*"). Um *programa* é uma implementação concreta do sistema.

A metodologia de Floyd-Hoare estabelece uma lógica de programas em que as fórmulas são da forma
$\{\phi\} S \{\psi\}$, sendo $\phi$ e $\psi$ predicados e $S$ um programa. $\phi$ representa as *pré-condições* (condições que se assumem estarem garantidas quando o programa inicia a sua execução) e $\psi$ representa as *pós-condições* (condições que têm que ser asseguradas quando o programa termina). Chama-se especificação ao par $(\phi,\psi)$ e *triplo de Hoare* à fórmula $\{\phi\} S \{\psi\}$.

A verificação formal de programas permite garantir que um programa se comporta de acordo com a sua especificação, seguindo a seguinte estratégia. Numa primeira fase (a *geração de condições de verificação*) transforma-se o programa e a sua especificação numa fórmula lógica, chamada *condição de verificação* (VC), que se for válida garante que o programa satisfaz a especificação. Numa segunda fase é usado um *SMT solver* para determinar a validade da VC (ou encontrar modelos que descrevem um traço do programa que conduz à violação da propriedade).
É assim uma técnica de análise estática de programas que permite detectar erros que, eventualmente, só seriam detectados em tempo de execução.


Vamos considerar uma linguagem imperativa simples, de variáveis de tipo inteiro, em que um programa não é mais do que uma sequência dos seguintes comandos e anotações:

`skip` | `x = e` | `if b then C1 then C2` | `while b do C` | `assume P`| `assert P`

A metodologia que vamos aplicar baseia-se na utilização de uma linguagem intermédia, que permita representação do programa e da sua especificação, e a partir da qual é fácil gerar a denotação lógica do programa.
Assim, o primeiro passo consiste na tradução do programa e da especificação para essa linguagem intermédia, e o segundo passo na transformação dessa representação do programa numa VC. 

A noção de *fluxo de programa* é fundamental para a caracterização de programas imperativos e está na base da linguagem intermédia que vamos usar e a que chamaremos *linguagem de fluxos*. 
Um fluxo é basicamente uma sequência gerada com base nas seguintes construções:
`skip` que corresponde à sequência vazia, `;` que acrescenta um comando a um fluxo, e `||` que permite criar um fluxo que corresponde a uma escolha não-determinista.

A construção `(Fluxo || Fluxo)` denota escolha não-determinista entre dois fluxos. 
Tipicamente estes fluxos estão encabeçados por um `assume P` e somente quando `P` for verdadeiro é que esse fluxo está qualificado para execução. 
Esta construção é usada na tradução da instrução `if b then C1 else C2` que é reescrita da seguinte forma:
`(assume b ; C1 || assume (not b) ; C2)`.

A tradução da instrução `while b do C` é mais complexa e será analizada mais abaixo.

Um triplo de Hoare $\{\phi\} S \{\psi\}$ pode ser representado por:  ${\sf assume }\: \phi\; ; S \; ; {\sf assert } \:\psi$.

Para a geração das VCs há duas técnicas standard:

- *Weakest pre-condition* (WPC): onde, dada a representação do programa e a pós-condição, é gerada a pré-condição mais fraca que é suficiente para garantir que quando o programa termina a pós-condição é assegurada.

- *Strongest post-condition* (SPC): onde, dada a representação do programa e a pré-condição, é gerada a pós-condição mais forte que é possível garantir quando o programa termina.



### Weakest pre-condition

A denotação `[C]` associa a cada fluxo `C` um predicado que o caracteriza a sua correcção em termos lógicos (a sua VC) segundo a técnica WPC, sendo calculada pelas seguintes regras.

$
\begin{array}{l}
[{\sf skip}] = True \\
[{\sf assume}\:\phi] = True \\
[{\sf assert}\:\phi] = \phi \\
[ x = e ] = True \\
[(C_1 || C_2)] = [C_1] \wedge [C_2] \\
\\
[{\sf skip}\, ; C] = [C] \\
[{\sf assume}\:\phi\, ; C] = \phi \to [C] \\
[{\sf assert}\:\phi\, ; C] = \phi \wedge [C] \\
[ x = e \, ; C] = [C][e/x] \\
[(C_1 || C_2)\, ; C] = [(C_1 ; C) || (C_2 ; C)]
\end{array}
$


Começamos por instalar o Z3.

In [ ]:
!pip install z3-solver

     |████████████████████████████████| 32.9 MB 41.6 MB/s 


### Exercício 1

Considere o seguinte programa anotado que faz o *swap* de duas variáveis usando operações aritméticas.
```python
assume x == a and y == b;
x = x + y;
y = x - y;
x = x - y;
assert x == b and y == a
```
Usando a abordagem WPC calcule a denotação lógica deste programa (a sua condição de verificação).

```python

[assume x == a and y == b; x = x + y; y = x - y; x = x - y; assert x == b and y == a]

<=>

(x == a and y == b) -> [x = x + y; y = x - y; x = x - y; assert x == b and y == a]

<=>

(x == a and y == b) -> [y = x - y; x = x - y; assert x == b and y == a][(x + y)/x]

<=>

(x == a and y == b) -> [x = x - y; assert x == b and y == a][(x - y)/y][(x + y)/x]

<=>

(x == a and y == b) -> [assert x == b and y == a] [(x - y)/x] [(x - y)/y] [(x + y)/x]

<=>

(x == a and y == b) -> (x == b and y == a) [(x - y)/x] [(x - y)/y] [(x + y)/x]


```

### Exercício 2

Complete definição da função `prove` que verifica a validade de uma fórmula lógica usando o Z3.

In [ ]:
from z3 import *

def prove(f):
    # completar
    s = Solver()
    s.add(Not(f))
    if s.check() == unsat:
      print("Provado")
    else:
      print("Não foi provado")

### Exercício 3

Use a função `prove` para verificar a condição de verificação obtida no exercício 1, assumindo que as variáveis são inteiros de tamanho ilimitado. Pode utilizar a função `substitute(a,(b,c))` para substituir em `a` todas as ocorrências de `b` por `c`.

In [ ]:
x, y, a, b = Ints("x y a b")
pre = And(x == a, y == b)
pos = And(x == b, y == a)
VC = Implies(pre, substitute(substitute(substitute(pos, (x, x - y)), (y, x - y)), (x, x + y)))

prove(VC)

Provado


### Exercício 4

Verifique que esta condição de verificação continua a ser válida mesmo que as variáveis sejam inteiros de tamanho limitado (por exemplo, com 32 bits).

In [ ]:
x, y, a, b = BitVecs("x y a b", 32)
pre = And(x == a, y == b)
pos = And(x == b, y == a)
VC = Implies(pre, substitute(substitute(substitute(pos, (x, x - y)), (y, x - y)), (x, x + y)))

prove(VC)

Provado


### Exercício 5

Considere o seguinte programa anotado que calcula o máximo de dois números.
```python
if x>y 
   then m = x
   else m = y;
assert m >= x and m >= y and (m == x or m == y)
```
Usando a abordagem WPC calcule a denotação lógica deste programa (a sua condição de verificação). Note que primeiro tem que traduzir o programa para a linguagem de fluxos. Use também o Z3 para verificar a sua validade.

```python

[(assume x > y; m = x || assume (not x > y); m = y); assert m >= x and m >= y and (m == x or m == y)]

[assume x > y; m = x; assert m >= x and m >= y and (m == x or m == y)]
||
[assume (not x > y); m = y; assert m >= x and m >= y and (m == x or m == y)]

<=>

x > y -> [m = x; assert m >= x and m >= y and (m == x or m == y)]
||
not x > y -> [m = y; assert m >= x and m >= y and (m == x or m == y)]

<=>

x > y -> [assert m >= x and m >= y and (m == x or m == y)] [x/m]
||
not x > y -> [assert m >= x and m >= y and (m == x or m == y)] [y/m]

<=>

x > y -> (m >= x and m >= y and (m == x or m == y)) [x/m]
||
not x > y -> (m >= x and m >= y and (m == x or m == y)) [y/m]
```

In [ ]:
x, y, m = Ints("x y m")

pre = x > y
pos = And(m >= x, m >= y, Or(m == x, m == y))
VC1 = Implies(pre, substitute(pos, (m, x)))
VC2 = Implies(Not(pre), substitute(pos, (m, y)))
VC = And(VC1, VC2)
prove(VC)

Provado


### Strongest post-condition

Na abordagem SPC a denotação de um fluxo com um comando de atribuição introduz um quantificador existencial, o que não é adequado à verificação com SMT solvers: 
$ \quad [ C \; ; x = e ] \; =  \; \exists a. (x = e[a/x]) \wedge [C][a/x] $

Para lidar com este problema pode-se converter o programa original ao formato "*single assignment*" (SA).
Num programa SA cada variável só pode ser usada depois de ser atribuida e só pode ser atribuída uma única vez.

Um programa (onde variáveis são atribuídas mais do que uma vez) pode ser reescrito num programa SA criando "clones" distintos das variáveis de forma a que seja possível fazer uma atribuição única a cada instância.

Neste caso, a denotação `[C]` associa a cada fluxo `C` um predicado que caracteriza a sua correcção em termos lógicos (a sua VC) segundo a técnica SPC, sendo calculada pelas seguintes regras.

$
\begin{array}{l}
[{\sf skip}] = True \\
[{\sf assume}\:\phi] = \phi \\
[{\sf assert}\:\phi] = \phi \\
[x = e ] = (x = e)\\
[(C_1 || C_2)] = [C_1] \vee [C_2] \\
\\
[C \, ; {\sf skip}\;] = [C] \\
[C \, ;{\sf assume}\:\phi] = [C] \wedge \phi \\
[C \, ;{\sf assert}\:\phi] = [C] \to \phi \\
[ C \, ; x = e ] = [C] \wedge (x = e) \\
[C\,; (C_1 || C_2)] = [(C ; C_1) || (C; C_2)]
\end{array}
$



### Exercício 6

Usando a abordagem SPC calcule a denotação lógica do programa que faz o *swap* de duas variáveis (a sua condição de verificação). Note que primeiro tem que traduzir o programa para a linguagem de fluxos SA. Use também o Z3 para verificar a validade da condição de verificação obtida.

```python
assume x == a and y == b;
x = x + y;
y = x - y;
x = x - y;
assert x == b and y == a
```

Versão SA:

```python
assume x0 == a and y0 == b
x1 = x0 + y0
y1 = x1 - y0
x2 = x1 - y1
assert x2 == b and y1 == a
```

```python
[assume x0 == a and y0 == b; x1 = x0 + y0; y1 = x1 - y0; x2 = x1 - y1; assert x2 == b and y1 == a]

<=>

[assume x0 == a and y0 == b; x1 = x0 + y0; y1 = x1 - y0; x2 = x1 - y1] -> x2 == b and y1 == a

<=>

[assume x0 == a and y0 == b; x1 = x0 + y0; y1 = x1 - y0] and x2 = x1 - y1 -> x2 == b and y1 == a

<=>

[assume x0 == a and y0 == b; x1 = x0 + y0] and y1 = x1 - y0 and x2 = x1 - y1 -> x2 == b and y1 == a

<=>

[assume x0 == a and y0 == b] and x1 = x0 + y0 and y1 = x1 - y0 and x2 = x1 - y1 -> x2 == b and y1 == a

<=>

(x0 == a and y0 == b and x1 = x0 + y0 and y1 = x1 - y0 and x2 = x1 - y1) -> x2 == b and y1 == a
```

In [ ]:
a, b, x0, x1, x2, y0, y1 = Ints("a b x0 x1 x2 y0 y1")
pre = And(x0 == a, y0 == b)
preImplies = And(pre, x1 == x0 + y0, y1 == x1 - y0, x2 == x1 - y1)
pos = And(x2 == b, y1 == a)
VC = Implies(preImplies, pos)
prove(VC)

Provado


### Exercício 7

Usando a abordagem SPC calcule a denotação lógica do programa (a sua condição de verificação) que calcula o máximo de 2 números. Note que primeiro tem que traduzir o programa para a linguagem de fluxos SA. Use também o Z3 para verificar a validade da condição de verificação obtida.

```python
if x>y 
   then m = x
   else m = y;
assert m >= x and m >= y and (m == x or m == y)
```

```python
if x>y 
   then m1 = x
   else m2 = y;
   m3 = (x > y) ? m1 : m2
assert m3 >= x and m3 >= y and (m3 == x or m3 == y)
```

Esta secção é estupida, o $m$ nunca tem mais que uma atribuição

### Programas iterativos

Na verificação de programas com ciclos a dificuldade está na verificação da validade do triplo de Hoare 
$\{\phi\} {\sf while} \; b \;{\sf do } \;C \{\psi\}$.

Uma primeira abordagem assenta na noção de *invariante*, uma propriedade que tem que ser válida à entrada do ciclo, ao final de cada iteração, e que deve ser suficientemente forte para à saida do ciclo garantir a pós-condição.

Esta ideia está sintetizada na regra:
$$
\frac{\{\phi\}{\sf skip} \{\theta\} \quad \{\theta \wedge b\} C \{\theta\} \quad \{\theta \wedge \neg b\}{\sf skip} \{\psi\} }{\{\phi\} {\sf while} \; b \;{\sf do } \;C \{\psi\}}
$$

Assim, para garantir a validade de $\{\phi\} {\sf while} \; b \;{\sf do } \;C \{\psi\}$ basta garantir a validade das premissas desta regra, gerando as VCs de cada um desses triplos (que correspondem à "inicialização", "preservação" e "utilidade"). Se as VCs forem válidas, então o triplo que está na conclusão é válido.

A principal dificuldade neste processo está na descoberta do invariante $\theta$ apropriado.

### Exercício 8

Considere o seguinte programa anotado (incluindo o invariante de ciclo) que calcula o máximo de um array de inteiros.

```python
assume n >= 1 and i == 1 and m == A[0]
while i<n:
    invariant i<=n and forall j . 0 <= j < i -> m >= A[j]
    if A[i]>m:
        m = A[i]
    i = i+1
assert forall j . 0 <= j < n -> m >= A[j]
```
Indique os fluxos correspondentes aos triplos de Hoare necessários para verificar a inicialização, preservação e utilidade deste invariante. Calcule também as respectivas condições de verificação.

```python
pre = n >= 1 and i == 1 and m == A[0]
pos = forall j . 0 <= j < n -> m >= A[j]
invariante = i<=n and forall j . 0 <= j < i -> m >= A[j]

init = [assume pre; skip; assert invariante]
= pre -> invariante

utilidade = [assume invariante and not i < n; skip; assert pos]
= (invariante and not i < n) -> pos

preservacao = [assume invariante and i < n; (assume A[i] > m; m = A[i] || assume not A[i] > m; skip); i = i + 1; assert invariante]
= (invariante and i < n) 
  -> [(assume A[i] > m; m = A[i] || assume not A[i] > m; skip); i = i + 1; assert invariante]
= (invariante and i < n) 
  -> ([assume A[i] > m; m = A[i]; i = i + 1; assert invariante] || [assume not A[i] > m; skip; i = i + 1; assert invariante])
= (invariante and i < n) 
  -> (A[i] > m -> [m = A[i]; i = i + 1; assert invariante] || not A[i] > m -> [skip; i = i + 1; assert invariante])
= (invariante and i < n) 
  -> (A[i] > m -> [i = i + 1; assert invariante] [A[i]/m] || not A[i] > m -> [i = i + 1; assert invariante])
= (invariante and i < n) 
  -> (A[i] > m -> invariante [A[i]/m] [(i+1)/i] || not A[i] > m -> invariante [(i+1)/i])
```

### Exercício 9

Utilize o Z3 para verificar a validade das condições de verificação obtidas no exercício 7. Utilize a teoria de arrays do Z3, descrita no seguinte manual https://ericpony.github.io/z3py-tutorial/advanced-examples.htm.

In [ ]:
# Não sei porque é que nao funciona
A = Array('A', IntSort(), IntSort())
m, i, n, j = Ints("m i n j")

pre = And(n >= 1, i == 1, m == A[0])
pos = ForAll(j, Implies(And(0 <= j, j < n), m >= A[j]))
invariante = And(i <= n, ForAll(j, Implies(And(j <= 0, j < i), m >= A[j])))

init = Implies(pre, invariante)

utilidade = Implies(And(invariante, Not(i < n)), pos)

preservacao1 = And(invariante, i < n)
preservacao2 = Implies(A[i] > m, substitute(substitute(invariante, (m, A[i])), (i, i+1)))
preservacao3 = Implies(Not(A[i] > m), substitute(invariante, (i, i+1)))
preservacao = Implies(preservacao1, And(preservacao2, preservacao3))

VC = And(init, utilidade, preservacao)
prove(VC)

Não foi provado
